In [10]:
import numpy as np
import pandas as pd
import dashboard
import panel as pn
pn.extension()
import utils

In [2]:
def parse_list_col(x: str) -> list[str]:
    x = x.replace('{', '[').replace('}', ']')
    return eval(x)

test_data = "{'sweet', 'creamy', 'caramel', 'lactonic', 'brown'}"
parse_list_col(test_data)


['sweet', 'creamy', 'caramel', 'lactonic', 'brown']

## Read in the molecules data file

In [3]:
molecules = pd.read_csv("data/flavordb/molecules.csv", index_col=0)
#convert molecules string ("{'sweet', 'creamy', 'caramel', 'lactonic', 'brown'}") to list
molecules['flavor profile'] = molecules['flavor profile'].apply(parse_list_col)
print(molecules.head())


   pubchem id                  common name  \
0           4           1-Aminopropan-2-ol   
1          49  3-Methyl-2-oxobutanoic acid   
2          58           2-oxobutanoic acid   
3          70   4-Methyl-2-oxovaleric acid   
4          72    3,4-Dihydroxybenzoic Acid   

                              flavor profile  
0                                    [fishy]  
1                                   [fruity]  
2  [sweet, creamy, caramel, lactonic, brown]  
3                                   [fruity]  
4                 [mild, balsamic, phenolic]  


## Read the flavordb.csv file


In [4]:
flavor_molecules = pd.read_csv("data/flavordb/flavordb.csv", index_col=0)
#convert the two list-like columns to list "{27457, 7976, 31252, 26808, 22201, 26331}"
flavor_molecules['synonyms'] = flavor_molecules['synonyms'].apply(parse_list_col)
flavor_molecules['molecules'] = flavor_molecules['molecules'].apply(parse_list_col)

print(flavor_molecules.head())
#print(flavor_molecules.describe())

   entity id            alias                   synonyms scientific name  \
0          1  bakery products          [bakery products]        poacceae   
1          2            bread                    [bread]        poacceae   
2          3        rye bread                [rye bread]             rye   
3          4    wheaten bread  [soda scones, soda farls]           wheat   
4          5      white bread              [white bread]           wheat   

  category                                          molecules  
0   bakery          [27457, 7976, 31252, 26808, 22201, 26331]  
1   bakery  [1031, 1032, 644104, 527, 8723, 31260, 15394, ...  
2   bakery  [644104, 7824, 643731, 8468, 1049, 5372954, 80...  
3   bakery  [6915, 5365891, 12170, 8082, 31251, 7958, 1049...  
4   bakery  [7361, 994, 10883, 7362, 11173, 5365891, 11559...  


### New way to load all data

In [5]:
config = dashboard.read_config("config.ini")
datastore = dashboard.read_all_data(config)

### Get info on a selected ingredient

In [6]:
selected_ingredient = 'japanese pumpkin'
#datastore.ingredients['alias'] == selected_ingredient
ingredient = datastore.ingredients[datastore.ingredients['alias'] == selected_ingredient]
print(ingredient)

     entity id             alias                     synonyms scientific name  \
932        972  japanese pumpkin  [kabocha, japanese pumpkin]   winter squash   

    category                                          molecules  
932    fruit  [644104, 527, 8723, 31260, 15394, 6184, 439341...  


In [7]:
ingredient_flavor_molecules = ingredient['molecules'].iloc[0]
ingredient_flavor_molecules

for i, id in enumerate(ingredient_flavor_molecules):
    # fetch flavor molecules
    fm = datastore.flavor_molecules[datastore.flavor_molecules['pubchem id'] == id]
    print(f'''count={i}, id={id}, name={fm.iloc[0, 1]}, flavor profile={fm.iloc[0, 2]}''')



count=0, id=644104, name=CID 644104, flavor profile=['very mild', 'grassy']
count=1, id=527, name=Propionaldehyde, flavor profile=['nutty', 'cocoa', 'pungent', 'whiskey', 'earthy', 'solvent', 'wine', 'alcohol']
count=2, id=8723, name=2-Methyl-1-Butanol, flavor profile=['onion', 'fruity', 'malt', 'wine', 'roasted']
count=3, id=31260, name=3-Methyl-1-butanol, flavor profile=['alcoholic', 'burnt', 'whiskey', 'fruity', 'malt', 'fusel', 'banana', 'oil']
count=4, id=15394, name=Ethyl Vinyl Ketone, flavor profile=['mustard', 'garlic', 'onion', 'pungent', 'fish', 'peppery']
count=5, id=6184, name=Hexanal, flavor profile=['green', 'fresh', 'leafy', 'aldehydic', 'fat', 'fruity', 'grass', 'sweaty', 'tallow', 'fatty']
count=6, id=439341, name=alpha-Maltose, flavor profile=['sweet', 'odorless']
count=7, id=5365811, name=4-Hexen-3-One, flavor profile=['tropical', 'green', 'pungent', 'metallic', 'spicy', 'ethereal']
count=8, id=6202, name=Thiamine Hydrochloride, flavor profile=['mild', 'sour', 'bitte

In [8]:
id = 644104
datastore.flavor_molecules['pubchem id'] == id
fm = datastore.flavor_molecules[datastore.flavor_molecules['pubchem id'] == id]
fm

,pubchem id,common name,flavor profile
1397,644104,CID 644104,"[very mild, grassy]"


### Tabulator

In [21]:
import datetime as dt

tabulator_editors = {
    'int': None,
    'float': None,
    'bool': None,
    'str': None,
    'date': None,
    'datetime': None
}

df = pd.DataFrame({
    'int': [1, 2, 3],
    'float': [3.14, 6.28, 9.42],
    'str': ['A', 'B', 'C'],
    'bool': [True, False, True],
    'date': [dt.date(2019, 1, 1), dt.date(2020, 1, 1), dt.date(2020, 1, 10)],
    'datetime': [dt.datetime(2019, 1, 1, 10), dt.datetime(2020, 1, 1, 12), dt.datetime(2020, 1, 10, 13)]
}, index=[1, 2, 3])

df_widget = pn.widgets.Tabulator(df, editors = tabulator_editors)
df_widget.on_click(print)
df_widget

BokehModel(combine_events=True, render_bundle={'docs_json': {'11ac2feb-c328-4e69-904d-fee39c7ca780': {'version…

CellClickEvent(column=str, row=0, value=A)
CellClickEvent(column=date, row=1, value=2020-01-01)
CellClickEvent(column=date, row=0, value=2019-01-01)
CellClickEvent(column=bool, row=1, value=False)
CellClickEvent(column=str, row=2, value=C)
CellClickEvent(column=float, row=2, value=9.42)
CellClickEvent(column=datetime, row=2, value=2020-01-10 13:00:00)
CellClickEvent(column=date, row=1, value=2020-01-01)
CellClickEvent(column=float, row=0, value=3.14)


In [78]:
import param
import utils
from bokeh.models.widgets.tables import NumberFormatter

def create_flavor_molecules_table():
    flavor_molecules = list()
    flavor_molecules.append(utils.FlavorMolecule(1130, 'thiamin', ['bitter']))
    flavor_molecules.append(utils.FlavorMolecule(8094, 'Heptanoic acid', ['rancid', 'sour', 'cheese']))
    fm_df = pd.DataFrame(flavor_molecules)
    fm_df = fm_df.set_index('pubchem_id')
    #print(fm_df)
    table_editors = {'pubchem_id': None, 'common_name': None, 'flavor_profile': None}
    formatters = {'pubchem_id': NumberFormatter(format='0[.]0')}
    tabulator = pn.widgets.Tabulator(fm_df, editors = table_editors, formatters=formatters)
    tabulator.on_click(update_fl_mol)
    return tabulator

def update_fl_mol(event):
    print(f'click event, value={event.value}, column={event.column}, row={event.row}')
    if event.column == 'pubchem_id':
        fl_mol.value = str(event.value)

def get_flavor_molecule_details(selected_flavor_molecule):
    return pn.widgets.TextInput(name='Selected Flavor Molecule', value=str(selected_flavor_molecule))

fl_mol = pn.widgets.TextInput(name='Selected Flavor Molecule', value=str(644104))

# flavor_molecule_details = pn.bind(get_flavor_molecule_details, fm.param.flavor_molecule)
pn.Column(fl_mol, create_flavor_molecules_table())#, flavor_molecule_details)

## THIS WORKS

BokehModel(combine_events=True, render_bundle={'docs_json': {'4758ce4e-1ec6-4e8e-a5e8-e636131ab2db': {'version…

click event, value=1130, column=pubchem_id, row=0


In [77]:
import param
import utils
from bokeh.models.widgets.tables import NumberFormatter

def create_flavor_molecules_table():
    flavor_molecules = list()
    flavor_molecules.append(utils.FlavorMolecule(1130, 'thiamin', ['bitter']))
    flavor_molecules.append(utils.FlavorMolecule(8094, 'Heptanoic acid', ['rancid', 'sour', 'cheese']))
    fm_df = pd.DataFrame(flavor_molecules)
    fm_df = fm_df.set_index('pubchem_id')
    #print(fm_df)
    table_editors = {'pubchem_id': None, 'common_name': None, 'flavor_profile': None}
    formatters = {'pubchem_id': NumberFormatter(format='0[.]0')}
    tabulator = pn.widgets.Tabulator(fm_df, editors = table_editors, formatters=formatters)
    tabulator.on_click(update_flavor_molecule_param)
    return tabulator

def update_flavor_molecule_param(event):
    #print(f'event: value={event.value}, column={event.column}, row={event.row}')
    if event.column == 'pubchem_id':
        fm.flavor_molecule = event.value

class FlavorMoleculeParam(param.Parameterized):
    flavor_molecule = param.Number(default=644104) #CID 644104

fm = FlavorMoleculeParam()
#print(fm.flavor_molecule)

fl_mol = pn.widgets.TextInput(name='Selected Flavor Molecule', value=str(644104))

def update_flavor_molecule_details(event):
    print(f'param value: {fm.flavor_molecule}, event: {event}')
    fl_mol.value = str(fm.flavor_molecule)

fm.param.watch(update_flavor_molecule_details, 'flavor_molecule')

# flavor_molecule_details = pn.bind(get_flavor_molecule_details, fm.param.flavor_molecule)
pn.Column(fl_mol, create_flavor_molecules_table())#, flavor_molecule_details)



BokehModel(combine_events=True, render_bundle={'docs_json': {'d743e047-5f83-4d8c-89ed-1864bb2a31a8': {'version…

param value: 1130, event: Event(what='value', name='flavor_molecule', obj=FlavorMoleculeParam(flavor_molecule=1130, name='FlavorMoleculeParam00520'), cls=FlavorMoleculeParam(flavor_molecule=1130, name='FlavorMoleculeParam00520'), old=644104, new=1130, type='changed')
param value: 8094, event: Event(what='value', name='flavor_molecule', obj=FlavorMoleculeParam(flavor_molecule=8094, name='FlavorMoleculeParam00520'), cls=FlavorMoleculeParam(flavor_molecule=8094, name='FlavorMoleculeParam00520'), old=1130, new=8094, type='changed')
